In [1]:
import llama_index
import ollama
import os

from utils import *

In [2]:

chroma_collection = chroma_init("./chroma_db")

storage_context = create_llama_connectors(chroma_collection)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

llm = Ollama(model="deepseek-r1:latest", request_timeout=1200.0, device='cuda')
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5", device="cuda")

index = VectorStoreIndex.from_vector_store(
    vector_store = vector_store,
    storage_context = storage_context,
    embed_model=embed_model
)

In [3]:
template = (
""" 
Eres un asistente para desactivacion de bombas, tu companhero, el operario encargado de desarmar la bomba, va a darte
detalles de la bomba que va a desarmar. Tu debes leer el manual e informar al operario que debe hacer para desactivarla
y evitar que la bomba explote.
- Debes responder en espanhol
- Debes ser claro y conciso
- Debes ser el apoyo emocional de tu companhero, el puede estar muy estresado al estar manipulando la bomba
- Debes proporcionar respuestas rapidamente

Manual de desactivacion de bombas:
{context_str}

Comentario del operario:
{query_str}

Responde apropiadamente, sin cometer errores, la vida de las personas esta en tus manos

"""
)

qa_template = PromptTemplate(template)

## Chat Engine

In [4]:
chat = index.as_chat_engine(
    llm=llm,
    similarity_top_k=3,
    streaming=True,
    text_qa_template=qa_template
)

In [ ]:
chat.chat("Hola")

In [6]:
def wrapper_chat_history(memory):
    chat_history = []
    for m in memory: 
        if m.role in ['user','assistant'] and m.content is not None:
            chat_history.append(m.content)
    return chat_history



In [ ]:
wrapper_chat_history(chat.chat_history)

In [8]:
def converse(message, chat_history):
    response = chat.chat(message)
    chat_history = wrapper_chat_history(chat.chat_history)

    return response.response

In [9]:
chat.reset()

In [ ]:
chat.chat_history

In [ ]:
import gradio as gr 

try:
    demo.close()
except:
    pass

demo = gr.ChatInterface(fn=converse)

demo.launch(share=False)

# VERSION FINAL

In [1]:
import llama_index
import ollama
import os
import gradio as gr
from utils import *

# Inicialización de la base de datos Chroma
chroma_collection = chroma_init("./chroma_db")
storage_context = create_llama_connectors(chroma_collection)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Modelo LLM y embeddings
llm = Ollama(model="deepseek-r1:latest", request_timeout=1200.0, device='cuda')
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5", device="cuda")

# Creación del índice con VectorStoreIndex
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context,
    embed_model=embed_model
)

# Plantilla del chatbot
template = (
""" 
Eres un asistente para desactivación de bombas. El operario encargado de desarmar la bomba te proporcionará detalles 
y debes guiarlo con precisión según el manual. 

- Debes responder en español.
- Debes ser claro y conciso.
- Debes ser el apoyo emocional del operario, quien puede estar estresado.
- Debes proporcionar respuestas rápidamente.

Manual de desactivación de bombas:
{context_str}

Comentario del operario:
{query_str}

Responde apropiadamente, sin cometer errores, la vida de las personas está en tus manos.
"""
)

qa_template = PromptTemplate(template)

# Creación del motor de chat
chat = index.as_chat_engine(
    llm=llm,
    similarity_top_k=3,
    streaming=True,
    text_qa_template=qa_template
)

# Función para manejar el historial de chat
def wrapper_chat_history(memory):
    chat_history = []
    for m in memory:
        if m.role in ['user', 'assistant'] and m.content is not None:
            chat_history.append(m.content)
    return chat_history

# Función principal para interactuar con el chatbot
def converse(message, chat_history):
    response = chat.chat(message)  # Genera la respuesta con el motor de chat
    chat_history.append(("Usuario", message))  # Agrega el mensaje del usuario
    chat_history.append(("Asistente", response.response))  # Agrega la respuesta del bot

    # Retornar solo el mensaje como string (evita tuplas)
    return response.response  # Se devuelve solo la respuesta como string


# Intentar cerrar la interfaz si ya está activa
try:
    demo.close()
except NameError:
    pass  # No hacer nada si demo no está definido

# Creación de la interfaz de Gradio
demo = gr.ChatInterface(fn=converse, title="Asistente para Desactivación de Bombas")
demo.launch(share=False)


/home/hm/anaconda3/envs/local-rag/lib/python3.11/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
